# Analysing Tartu Smart Bike usage data 2019-2021

In [1]:
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd

In [2]:
data2019 = pd.read_csv("2019.csv", sep=',', header=0)
print("2019: ",data2019.shape)
data2020 = pd.read_csv("2020.csv", sep=',', header=0)
print("2020: ",data2020.shape)
data2021 = pd.read_csv("2021.csv", sep=',', header=0)
print("2021: ",data2021.shape)

2019:  (742559, 12)
2020:  (874032, 12)
2021:  (85004, 12)


In [3]:
#data2019 = data2019.drop(columns=['startstationserialnumber','endstationserialnumber'])
data2019 = data2019.drop(data2019[(data2019.startstationname == 'Warehouse') | (data2019.endstationname == 'Warehouse')].index)
data2019 = data2019.drop(data2019[(data2019.startstationname == 'Lauluväljak') | (data2019.endstationname == 'Lauluväljak')].index)
data2019 = data2019.drop(data2019[(data2019.startstationname == 'SPARK Demo') | (data2019.endstationname == 'SPARK Demo')].index)
data2019 = data2019.drop(data2019[(data2019.startstationname == 'Töökoda') | (data2019.endstationname == 'Töökoda')].index)
data2019 = data2019.drop(data2019[(data2019.startstationname == "Tour d'ÖÖ") | (data2019.endstationname == "Tour d'ÖÖ")].index)
data2019 = data2019.drop(data2019[(data2019.startstationname == "-- Määramata --") | (data2019.endstationname == "-- Määramata --")].index)
data2019 = data2019.drop(data2019[(data2019.startstationname == "Metallica parkla") | (data2019.endstationname == "Metallica parkla")].index)
data2019 = data2019.drop(data2019[(data2019.startstationname == "Undetermined") | (data2019.endstationname == "Undetermined")].index)
data2019 = data2019.drop(data2019[data2019['length'] < 0.05].index)

data2019 = data2019.drop(data2019[data2019.length == 0].index)

def fun():
    if (first3IdNumber[0] == 3 | first3IdNumber[0] == 4):
        data2019['yearOfBirth'] = int('19'+first3IdNumber[1:])
    elif(first3IdNumber[0] == 5 | first3IdNumber[0] == 6):
        data2019['yearOfBirth'] = int('20'+first3IdNumber[1:])
#data2019[['yearOfBirth', 'first3IdNumber']].where(data2019[(data2019['yearOfBirth'][2:] == data2019['first3IdNumber'][1:])], df[['yearOfBirth', 'first3IdNumber']].values)
#data2019[['yearOfBirth', 'first3IdNumber']] - vali mitu tulpa korraga
data2019['yearOfBirth'] = data2019['yearOfBirth'].fillna(0)
data2019['yearOfBirth'] = data2019['yearOfBirth'].astype(str)
data2019[['yearOfBirth', 'first3IdNumber']]
data2019.dtypes

cyclenumber                   int64
unlockedat                   object
unlockedattime               object
lockedat                     object
lockedattime                 object
startstationserialnumber     object
startstationname             object
endstationserialnumber       object
endstationname               object
length                      float64
yearOfBirth                  object
first3IdNumber               object
dtype: object

In [ ]:
#data2020 = data2020.drop(columns=['startstationserialnumber','endstationserialnumber'])
data2020 = data2020.drop(data2020[(data2020.startstationname == '-- Määramata --') | (data2020.endstationname == '-- Määramata --')].index)
data2020 = data2020.drop(data2020[(data2020.startstationname == 'Rahu 8') | (data2020.endstationname == 'Rahu 8')].index)
data2020 = data2020.drop(data2020[(data2020.startstationname == 'SPARK Demo') | (data2020.endstationname == 'SPARK Demo')].index)
data2020 = data2020.drop(data2020[(data2020.startstationname == 'Töökoda') | (data2020.endstationname == 'Töökoda')].index)
data2020 = data2020.drop(data2020[(data2020.startstationname == 'Undetermined') | (data2020.endstationname == 'Undetermined')].index)
data2020 = data2020.drop(data2020[(data2020.startstationname == 'Ihaste Päev') | (data2020.endstationname == 'Ihaste Päev')].index)
data2020 = data2020.drop(data2020[data2020['length'] < 0.05].index)
data2020 = data2020.drop(data2020[data2020.length == 0].index)


data2020

In [ ]:
#data2021 = data2021.drop(columns=['startstationserialnumber','endstationserialnumber'])
data2021 = data2021.drop(data2021[(data2021.startstationname == 'SPARK Demo') | (data2021.endstationname == 'SPARK Demo')].index)
data2021 = data2021.drop(data2021[(data2021.startstationname == '-- Määramata --') | (data2021.endstationname == '-- Määramata --')].index)
data2021 = data2021.drop(data2021[(data2021.startstationname == 'Töökoda') | (data2021.endstationname == 'Töökoda')].index)
data2021 = data2021.drop(data2021[(data2021.startstationname == 'Undetermined') | (data2021.endstationname == 'Undetermined')].index)
data2021 = data2021.drop(data2021[data2021['length'] < 0.05].index)
data2021 = data2021.drop(data2021[data2021.length == 0].index)
data2021

In [ ]:
andmed19end = dict(data2019['endstationname'].value_counts())
andmed19sta = dict(data2019['startstationname'].value_counts())

andmed21end = dict(data2021['endstationname'].value_counts())
andmed21end

andmed20end = dict(data2020['endstationname'].value_counts())
andmed21end
# for i in andmed:
#     print(andmed[i])

In [ ]:
# Faili salvestamine

#data2019.to_csv('data19cha.csv', index=False)
#data2020.to_csv('data20cha.csv', index=False)
#data2021.to_csv('data21cha.csv', index=False)